In [1]:
from transitions.extensions import LockedHierarchicalMachine as LHMachine
from transitions import State
import threading

In [2]:
DIALOG_STATES = [
    'not_initialized', 
    'idle', 
    {
        'name': 'activated', 
        'initial': 'preparing', 
        'children': ['preparing', 'listening', 'processing'],
    },
    'aborted',
    'loading',
    'quit',
]

# states machine transitions
# 
DIALOG_TRANSITIONS = [
    # initializing
    {
        'trigger': 'ready', 
        'source': 'not_initialized', 
        'dest': 'idle', 
        'prepare': None,
        'conditions': None, 
        'unless': None,
        'before': None,
        'after': None,
    },
    # activating
    {
        'trigger': 'ptt', 
        'source': 'idle', 
        'dest': 'activated', 
        'prepare': None,
        'conditions': None, 
        'unless': None,
        'before': None,
        'after': None,
    },
    {
        'trigger': 'enter_session', 
        'source': 'activated', 
        'dest': 'activated_listening', 
        'prepare': None,
        'conditions': None, 
        'unless': None,
        'before': None,
        'after': None,
    },
    {
        'trigger': 'result_returned', 
        'source': 'activated_listening', 
        'dest': 'activated_processing', 
        'prepare': None,
        'conditions': None, 
        'unless': None,
        'before': None,
        'after': None,
    },
    # return idle
    {
        'trigger': 'leave_session', 
        'source': 'aborted', 
        'dest': 'idle', 
        'prepare': None,
        'conditions': None, 
        'unless': None,
        'before': None,
        'after': None,
    },
    {
        'trigger': 'leave_session', 
        'source': 'activated_processing', 
        'dest': 'idle', 
        'prepare': None,
        'conditions': None, 
        'unless': None,
        'before': None,
        'after': None,
    },
    # aborting
    {
        'trigger': 'abort', 
        'source': 'activated', 
        'dest': 'aborted', 
        'prepare': None,
        'conditions': None, 
        'unless': None,
        'before': None,
        'after': None,
    },
    # loading
    {
        'trigger': 'leave_loding', 
        'source': 'loading', 
        'dest': 'idle', 
        'prepare': None,
        'conditions': None, 
        'unless': None,
        'before': None,
        'after': None,
    },
    {
        'trigger': 'load_data', 
        'source': 'idle', 
        'dest': 'loading', 
        'prepare': None,
        'conditions': None, 
        'unless': None,
        'before': None,
        'after': None,
    },
    # quiting
    {
        'trigger': 'quit', 
        'source': 'idle', 
        'dest': 'quit', 
        'prepare': None,
        'conditions': None, 
        'unless': None,
        'before': None,
        'after': None,
    },
]


In [3]:
class StateManger(LHMachine):
    
    _instance = None
    def __new__(cls, *args, **kw):
        
        if cls._instance is None:
            StateManger._instance = object.__new__(cls, *args, **kw)

        return cls._instance

    def __init__(self, *args, **kw):
        states = kw.get('states', [])
        transitions = kw.get('transitions', [])
        super(StateManger, self).__init__(states=states, transitions=transitions, initial='not_initialized')
        self.cond = threading.Condition()
        # self.state indicates the current state

    # def prepare_event(self):
    #     '''executed once before individual transitions are processed'''
    #     pass

    # def finalize_event(self):
    #     '''callbacks will be executed even if no transition took place or an exception has been raised'''
    #     pass

    # def on_enter_idle(self):
    #     pass

    # def on_enter_quit(self):
    #     pass

    def _is_matched_status(self, st):
        return self.state == st

    def need_state(self, status):
        # wait the stat
        # then leverage the decorated func
        def _func(func):
            def _inner(*args, **kw):
                self.cond.acquire()
                while not self._is_matched_status(status):
                    self.cond.wait()
                r = func(*args, **kw)
                self.cond.release()
                return r
            return _inner
        return _func

In [8]:
stm = StateManger(states=DIALOG_STATES,
            transitions=DIALOG_TRANSITIONS)
              

In [9]:
stm.state

'not_initialized'

In [10]:
vars(stm)

{'_after_state_change': [],
 '_before_state_change': [],
 '_buffered_transitions': [],
 '_finalize_event': [],
 '_initial': 'not_initialized',
 '_locked': 0,
 '_prepare_event': [],
 '_queued': False,
 '_transition_queue': deque([]),
 'abort': <functools.partial at 0x5fa4b10>,
 'auto_transitions': True,
 'cond': <Condition(<_RLock owner=None count=0>, 0)>,
 'enter_session': <functools.partial at 0x5fda1b0>,
 'events': {'abort': <LockedNestedEvent('abort')@100492272>,
  'enter_session': <LockedNestedEvent('enter_session')@100492080>,
  'leave_loding': <LockedNestedEvent('leave_loding')@100492368>,
  'leave_session': <LockedNestedEvent('leave_session')@100492208>,
  'load_data': <LockedNestedEvent('load_data')@100492432>,
  'ptt': <LockedNestedEvent('ptt')@100492016>,
  'quit': <LockedNestedEvent('quit')@100492496>,
  'ready': <LockedNestedEvent('ready')@100491696>,
  'result_returned': <LockedNestedEvent('result_returned')@100492144>,
  'to_aborted': <LockedNestedEvent('to_aborted')@1003

In [11]:
stm.to_quit()

True

In [12]:
stm.state

'quit'

In [13]:
stm.to_not_initialized()

True

In [14]:
stm.state 

'not_initialized'

In [15]:
stm.quit()

MachineError: "Can't trigger event quit from state not_initialized!"